In [ ]:
import numpy as np
import pandas as pd
import random
import math
import csv

In [ ]:
# Massive Data Set
csv_data = pd.read_csv("Danish_Atm_Data.csv")

In [ ]:
# Categorical variables to numerical
csv_data['month'] = csv_data['month'].astype('category').cat.codes
csv_data['day'] = csv_data['day'].astype('category').cat.codes
csv_data['year'] = csv_data['year'].astype('category').cat.codes
csv_data['hour'] = csv_data['hour'].astype('category').cat.codes
csv_data['atm_id'] = csv_data['atm_id'].astype('category').cat.codes
csv_data['weekday'] = csv_data['weekday'].astype('category').cat.codes
csv_data['atm_status'] = csv_data['atm_status'].astype('category').cat.codes
csv_data['atm_manufacturer'] = csv_data['atm_manufacturer'].astype('category').cat.codes
csv_data['atm_location'] = csv_data['atm_location'].astype('category').cat.codes
csv_data['atm_streetname'] = csv_data['atm_streetname'].astype('category').cat.codes
csv_data['atm_street_number'] = csv_data['atm_street_number'].astype('category').cat.codes
csv_data['atm_zipcode'] = csv_data['atm_zipcode'].astype('category').cat.codes
csv_data['atm_lat'] = csv_data['atm_lat'].astype('category').cat.codes
csv_data['atm_lon'] = csv_data['atm_lon'].astype('category').cat.codes
csv_data['currency'] = csv_data['currency'].astype('category').cat.codes
csv_data['card_type'] = csv_data['card_type'].astype('category').cat.codes
csv_data['service'] = csv_data['service'].astype('category').cat.codes
csv_data['message_code'] = csv_data['message_code'].astype('category').cat.codes
csv_data['message_text'] = csv_data['message_text'].astype('category').cat.codes
csv_data['Identification Number'] = csv_data['Identification Number'].astype('category').cat.codes

# Filling missing data
csv_data['message_code'].fillna(csv_data['message_code'].mode()[0], inplace=True)
csv_data['message_text'].fillna(csv_data['message_text'].mode()[0], inplace=True)
csv_data['Identification Number'].fillna(csv_data['Identification Number'].mode()[0], inplace=True)

#csv_data.dtypes
#csv_data.head()

In [ ]:
# Randomly choosing 1000 reference points from the CSV data
reference_points = csv_data.sample(1000)
reference_array = np.array(reference_points.index.to_numpy())
#print('reference object index',reference_array)
#pd_frame = pd.DataFrame(reference_points)

In [ ]:
# Calculating reference and object distance based on Permutation based Indexing
ref_object_distance = {}

for index, rows in csv_data.iterrows():
    data_array = rows.to_numpy()
    distance_list = []
    for reference_index,reference in reference_points.iterrows():
        ref_array = reference.to_numpy()
        #print(ref_array)
        distance = abs(np.sum(np.subtract(data_array,ref_array)))
        distance_list.append([reference_index,distance])
   
    ref_list_sorted_distance = sorted(distance_list,key=lambda x :x[1])
    reference , distance = zip(*ref_list_sorted_distance)
    ref_object_distance[index].append(reference[:10])           #pruned list with n_tilda =10
#print(ref_object_distance)

In [ ]:
#MIF file creation
mif = {}
for key,val in ref_object_distance.items():
    for j in reference_array:
        if j in ref_object_distance[key][0]:
            i = ref_object_distance[key][0].index(j)
            mif[str(j)].append(str(key) + ","+ str(i))
print(mif)
#print(mif, file=open("outputmiffinal.txt", "a"))

In [ ]:
#Similarity search on the MIF file

In [ ]:
#query object creation
query_input = csv_data.iloc[585]         #query input to be supplied to get nearest objects
#print (query_input)
queryobjectarray = np.array(query_input) #query object from the given data set
#print(queryobjectarray)
queryobjectdict = {}

for i,ref in enumerate(reference_array):
    ref_array = np.array(csv_data.iloc[ref])
    distance = abs(np.sum(np.subtract(queryobjectarray,ref_array)))
    queryobjectdict[ref] = distance       #dictionary of reference and distance

#print(queryobjectdict) #gives me reference and the distance

#gives me the distance of references
query_object_distance = sorted(queryobjectdict.items() , key=lambda x :x[1])
#print(query_object_distance)
ref_list = []
for key,value in query_object_distance: #key gives me the reference
    ref_list.append(key)
#print(ref_list)

In [ ]:
#accumulator logic for getting similar objects to query (objects with small accumulator value)
distance = {}
for ref in ref_list:                             # key from query_object_distance calculated above
    reference_list = list(mif[str(ref)])         # mif dictionary that was created above
    for i,j in enumerate(reference_list):        # ref_list has r1:(object,position)
        objectindex,position = j.split(',')
        distance[objectindex] = 0
        distance[objectindex] = distance[objectindex] + abs(ref_list.index(ref) - int(position))
    accumlator = sorted(distance.items(),key=lambda x:x[1])   #object close to given query    
print(ref[:20])                                               #20 nearest objects to the query